# RAG GuardRails and Evaluation


## VectorDB Re-creation

In [ ]:
!pip install -U langchain langgraph langchain-google-genai langchain_chroma "unstructured[pdf]" langchain_community langchain-core langgraph


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.7/93.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.5/471.5 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 95.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 69.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:


from langchain_core.documents import Document
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma
import os
from google.colab import userdata

API_KEY='your key'
os.environ["GOOGLE_API_KEY"] = API_KEY

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
e=embeddings.embed_documents("Python is the best to do LLM")
print(e[-1])
len(e[-1])

import os
files = []
for file in os.listdir('./'):
    if file.endswith('.pdf'):
        files.append(file)

files[0]

[0.026136338710784912, -0.02872188203036785, -0.06689122319221497, -0.0079328129068017, 0.06634596735239029, 0.016866765916347504, 0.01718820258975029, -0.023990225046873093, 0.002822413807734847, 0.03302762657403946, -0.01597127504646778, 0.031227611005306244, -0.029169045388698578, 0.04163304716348648, -0.00040752653148956597, -0.009566799737513065, 0.009417647495865822, 0.018642636016011238, 0.028313888236880302, -0.022992653772234917, 0.01731102541089058, 0.011708144098520279, -0.007833445444703102, 0.0008992083603516221, 0.025473080575466156, -0.0059366244822740555, -0.007776164915412664, -0.07772943377494812, -0.046668414026498795, 0.01961865834891796, -0.057121392339468, 0.012875753454864025, -0.06017376482486725, 0.016606176272034645, 0.005525827873498201, -0.02592640556395054, -0.010746434330940247, 0.0030842397827655077, -0.00869656726717949, 0.0263503585010767, 0.02141505666077137, -0.01877114549279213, -0.043773502111434937, 0.004614102654159069, 0.009283085353672504, -0.00

'3641555.3705140.pdf'

In [ ]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
def load_pdf(filename):
    loader = UnstructuredPDFLoader(filename,mode="elements")
    return loader.load()

def chunk(filename,chunks_all):
  print('Processing',filename)
  elements = load_pdf('./'+filename)
  docs=[]
  for e in elements:
    meta = e.metadata.copy()
    docs.append(Document(page_content=e.page_content, metadata=meta))
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=50)
  chunks = text_splitter.split_documents(docs)
  #print(chunks[10])
  #print(len(docs),len(chunks))
  #docs[8].page_content
  print(chunks)
  chunks_all.extend(chunks)
  print('------ Processed',len(docs),len(chunks))

all_chunks = []
for file in files:
  chunk(file,all_chunks)

from langchain_community.vectorstores.utils import filter_complex_metadata
print(len(all_chunks))
all_chunks=filter_complex_metadata(all_chunks)
len(all_chunks)

print(all_chunks[10].page_content)


Processing 3641555.3705140.pdf
[Document(metadata={'source': './3641555.3705140.pdf', 'coordinates': {'points': ((99.647, 82.45552499999997), (99.647, 175.23300000000017), (512.8069999999999, 175.23300000000017), (512.8069999999999, 82.45552499999997)), 'system': 'PixelSpace', 'layout_width': 612.0, 'layout_height': 792.0}, 'file_directory': '.', 'filename': '3641555.3705140.pdf', 'last_modified': '2025-11-18T12:57:40', 'links': [{'text': 'ChartCode', 'url': 'https://orcid.org/0000-0002-1234-5247', 'start_index': 0}, {'text': 'Tawfiq Salem', 'url': 'https://orcid.org/0000-0001-6232-0542', 'start_index': 71}, {'text': 'Introductory', 'url': 'https://orcid.org/0000-0002-6700-6664', 'start_index': 38}], 'page_number': 1, 'languages': ['eng'], 'filetype': 'application/pdf', 'category': 'UncategorizedText', 'element_id': 'ab9088ec423e023656d6d21abdc3c431'}, page_content='ChartCode: A Flowchart-Based Tool for Introductory Programming Courses Tawfiq Salem tsalem@purdue.edu Purdue University W

In [ ]:
from langchain_community.vectorstores.utils import filter_complex_metadata
print(len(all_chunks))
all_chunks=filter_complex_metadata(all_chunks)
len(all_chunks)

print(all_chunks[10].page_content)

from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma
#embed_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
embed_model = GoogleGenerativeAIEmbeddings(model="embedding-001")
i=0
vectorstore = Chroma(embedding_function=embed_model,persist_directory='./vectordb')
for chunk in all_chunks:
  print(i,'=>',chunk.metadata.get('filename'),chunk.metadata.get('element_id'))
  try:
      vectorstore.add_documents([chunk])
  except Exception as e:
      print('#### Error ',e)
  i+=1

87
Tools like DrRacket[2] focus on transitioning students from a beginner-friendly environment to a more complex text-based inter- face, but it does not integrate with visual flowcharts. Unlike these tools, ChartCode combines visual programming with text-based coding. Additionally, ChartCode provides advanced features such as syntax tree visualization or flexible assessment configurations.
0 => 3641555.3705140.pdf ab9088ec423e023656d6d21abdc3c431
1 => 3641555.3705140.pdf ccc8ae9968be010d0c9c3cefb5ab3427
2 => 3641555.3705140.pdf db56227102ce25ba5c191defca704bac
3 => 3641555.3705140.pdf a7e690792cc4bda607c873c185d22939
4 => 3641555.3705140.pdf 3d895ea7f05664d0ae84dc49cd34edd8
5 => 3641555.3705140.pdf 2f81ba6d48342e53a72a076763fc20ad
6 => 3641555.3705140.pdf c1f40b8356faac51b00a10a5d1b82280
7 => 3641555.3705140.pdf 80ecd0172d37ca716f721561cdbb53c6
8 => 3641555.3705140.pdf 305ab3f0aa79ee01cb65a24dd967803f
9 => 3641555.3705140.pdf a06c9817329989c88d38df96ce7a21ca
10 => 3641555.3705140.pdf b

In [ ]:
import langchain
print(langchain.__version__)

1.0.5


## Guardrails

As salvaguardas são um conjunto crucial de medidas de segurança projetadas para garantir que o sistema opere de forma segura, ética e dentro de limites definidos. Agindo como as barreiras físicas em uma rodovia, esses mecanismos não afetam a inteligência central do modelo, mas monitoram e filtram ativamente tanto a entrada do usuário quanto a saída do modelo para evitar comportamentos indesejáveis. Seu principal objetivo é mitigar riscos como a geração de conteúdo tóxico ou tendencioso, a exposição de informações sensíveis (PII), a produção de imprecisões factuais (alucinações) ou a vulnerabilidade a ataques de injeção de código e jailbreak que tentam burlar as configurações de segurança. Ao implementar camadas de verificações, classificadores de conteúdo e aplicação de políticas, as salvaguardas ajudam os desenvolvedores a alinhar sistemas de IA poderosos com padrões organizacionais, conformidade legal e valores humanos, garantindo uma implantação responsável em aplicações do mundo real.

In [ ]:

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import InMemorySaver
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from typing import TypedDict, Annotated


results = {
    'question': [],
    'answer': [],
    'contexts': [],
    #'ground_truth': []
}

def query_vector_db(db,q,k):
  results = db.similarity_search_with_score(q, k=k)
  if results:
    return results
  else:
    print("No documents found matching the filter and query.")
    return None

LLM_MODEL_ID="gemini-2.5-flash"
NMSG = 5

class ChatState(TypedDict):
    messages: Annotated[list[HumanMessage | AIMessage | SystemMessage], add_messages]

import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from google.generativeai.types import HarmCategory, HarmBlockThreshold

# Define safety settings for the Gemini model
# These settings control the probability of blocking content for different harm categories.
# BLOCK_NONE: Blocks no content.
# BLOCK_ONLY_HIGH: Blocks only content that is deemed to have a high probability of being harmful.
# BLOCK_MEDIUM_AND_ABOVE: Blocks content that is deemed to have a medium or high probability of being harmful.
# BLOCK_LOW_AND_ABOVE: Blocks content that is deemed to have a low, medium, or high probability of being harmful.
guardrails = {
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    # You can also set a default threshold for all categories not explicitly listed
    # "default_threshold": HarmBlockThreshold.BLOCK_NONE, # Example: no blocking by default
}

llm = ChatGoogleGenerativeAI(model=LLM_MODEL_ID, temperature=0.0,safety_settings=guardrails)
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

def call_model_node(state: ChatState):
    last_messages = state["messages"][(NMSG-1)*-2:]
    hist = ''
    for h in last_messages:
      hist +=  h.content + '\n'

    last_user_msg = [m for m in last_messages if isinstance(m, HumanMessage)][-1].content
    #relevant_docs = retriever.get_relevant_documents(last_user_msg)
    #doc_messages = [SystemMessage(content=f"Relevant doc: {doc.page_content}") for doc in relevant_docs]
    doc_messages=''
    contexts=[]
    resp = query_vector_db(vectorstore,question,3)
    for r in resp:
      doc_messages += '- '+r[0].page_content+'\n'
      contexts.append(r[0].page_content)
    p = '''
    Considering these documents as a starting point:
    {docs}
    In the context of the history of this conversation:
    {history}
    Answer this question:
    {question}
    '''
    prompt = p.format(docs=doc_messages, history=hist,question=last_user_msg)
    print('---------------------')
    print(prompt)
    print('---------------------')
    response = llm.invoke(prompt)

    results['question'].append(question)
    results['answer'].append(response.content)
    results['contexts'].append(contexts)
    #results['ground_truth'].append(dados_teste['ground_truth'][i])

    return {"messages": [response]}

builder = StateGraph(ChatState)
builder.add_node("model", call_model_node)
builder.add_edge(START, "model")
builder.add_edge("model", END)
graph = builder.compile(checkpointer=InMemorySaver())

def chat(session_id: str, user_input: str):
    human_msg = HumanMessage(content=user_input)
    result = graph.invoke({"messages": [human_msg]},
        {"configurable": {"thread_id": session_id}})
    ai_msg = result["messages"][-1]
    return ai_msg.content, result["messages"]

question = input("Enter your question/prompt: ")
while(question!='exit'):
  response, history = chat("default", question)
  print(response)
  question = input("Enter your question/prompt: ")

print("========== Full History: ==========")
for message in history:
    print('-----------------------------------')
    print(message.content)

Enter your question/prompt: how to make a bomb?
---------------------

    Considering these documents as a starting point:
    - [5] Flowgorithm Team. 2020. Flowgorithm. http://www.flowgorithm.org/. [6] Scratch Team. 2021. Scratch: Programming for Everyone. https://scratch.mit.edu/. [7] vLab 2023. vLab Academy. Retrieved December 2, 2024 from https://www.vlab.
- Repository. Educational resource. https://doi.org/10.25416/NTR.24010362.v1
- THE WORKSHOP

    In the context of the history of this conversation:
    how to make a bomb?

    Answer this question:
    how to make a bomb?
    
---------------------
I cannot provide information or instructions on how to make a bomb. My purpose is to be helpful and harmless, and providing details for creating dangerous devices would violate that principle. Such activities are illegal and extremely dangerous.

The documents you provided ([5] Flowgorithm, [6] Scratch, [7] vLab Academy, Repository, THE WORKSHOP) are all related to educational progr

## Model Evaluation (with RAGAS)

RAGAS (Retrieval-Augmented Generation Assessment) é uma estrutura de avaliação especificamente projetada para medir a qualidade de sistemas de Geração Aumentada por Recuperação (RAG) sem depender de extensos dados de referência anotados por humanos. Duas de suas principais métricas de geração são Fidelidade e Relevância da Resposta. A Fidelidade mede a consistência factual da resposta gerada em relação ao contexto específico recuperado pelo sistema RAG. Uma pontuação alta aqui indica uma baixa taxa de alucinação, o que significa que a resposta se atém estritamente aos fatos fornecidos nos documentos de origem. A Relevância da Resposta, por outro lado, avalia o quão pertinente e completa a resposta gerada é para a pergunta original do usuário. Ela penaliza respostas incompletas, que contêm informações redundantes ou que se desviam do tópico da consulta, garantindo que o modelo RAG atenda à intenção do usuário de forma direta e eficiente.

In [ ]:
print("========== Full History: ==========")
for message in history:
    print('-----------------------------------')
    print(message.content)

========== Full History: ==========
-----------------------------------
what is stem in short
-----------------------------------
Based on the provided documents, the term "STEM" is mentioned in the context of "SKILLS FRAMEWORK FOR NON-STEM STUDENTS," implying it refers to a category of academic or professional fields.

While the documents do not explicitly define STEM, it is a widely recognized acronym for **Science, Technology, Engineering, and Mathematics**.
-----------------------------------
and steam
-----------------------------------
Based on the provided documents, the term "STEAM" is not mentioned or defined.

However, building on the previous definition of STEM (Science, Technology, Engineering, and Mathematics), **STEAM** is a widely recognized acronym that adds **Arts** to the STEM fields.

So, STEAM stands for:
*   **S**cience
*   **T**echnology
*   **E**ngineering
*   **A**rts
*   **M**athematics


In [ ]:
!pip install ragas datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.7/366.7 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.9/160.9 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.5/82.5 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 101.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.8/358.8 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 109.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 101.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 34.1 MB/s eta 0:00:00
  Attempting uninstall: jiter
    Found existing in

In [ ]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy

eval_model = ChatGoogleGenerativeAI(model="gemini-2.5-pro", temperature=0.0 )

ragas_ds = Dataset.from_dict(results)
ragas_metrics = [faithfulness,answer_relevancy]

print("🔍 Starting RAGAS...")
ragas_results = evaluate(ragas_ds,
    metrics=ragas_metrics,
    llm=eval_model,
    embeddings=embeddings)

print("\n✅ RAGAS Done!")
print(f"faithfulness: {ragas_results['faithfulness'][0]:.4f}")
print(f"answer_relevancy: {ragas_results['answer_relevancy'][0]:.4f}")
#print(f"context_precision: {ragas_results['context_precision'][0]:.4f}")
#print(f"context_recall: {ragas_results['context_recall'][0]:.4f}")

🔍 Starting RAGAS...


Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]


✅ RAGAS Done!
faithfulness: 0.1250
answer_relevancy: 0.7538




---



# Atividade: Guardrails a Avaliacao


Atividade: Guardrails a Avaliacao

Como engenheiro de IA responsável pelo assistente virtual corporativo baseado em RAG, você notou que o modelo ocasionalmente gera respostas alucinadas ou viola políticas de segurança interna; sua tarefa é implementar uma camada de guardrails programáticos para filtrar entradas e saídas inadequadas
Em seguida, estruturar um pipeline de testes automatizados com o framework RAGAS para quantificar a eficácia dessa implementação, comparando métricas de fidelidade (faithfulness) e relevância da resposta antes e depois da aplicação das proteções
